#### Pre and per-item validators

Validators can do a few more complex things.

In [1]:
from typing import List
from pydantic import BaseModel, ValidationError, validator

In [2]:
class DemoModel(BaseModel):
    square_numbers: List[int] = []
    cube_numbers: List[int] = []
    
    # '*' is the same as 'cube_numbers', 'square_numbers' here:
    @validator("*", pre=True)
    def split_str(cls, v):
        if isinstance(v, str):
            return v.split("|")
        return v
    
    @validator("cube_numbers", "square_numbers")
    def check_sum(cls, v):
        if sum(v) > 42:
            raise ValueError("sum of numbers greater than 42")
        return v
    
    @validator("square_numbers", each_item=True)
    def check_squares(cls, v):
        assert v ** 0.5 % 1 == 0, f"{v} is not a square number"
        return v
    
    @validator("cube_numbers", each_item=True)
    def check_cubes(cls, v):
        # 64 ** (1 / 3) == 3.9999999999999996 (!)
        # this is not a good way of checking cubes
        assert v ** (1 / 3) % 1 == 0, f"{v} is not a cubed number"
        return v

In [3]:
print(f"{DemoModel(square_numbers=[1, 4, 9]) = }")

DemoModel(square_numbers=[1, 4, 9]) = DemoModel(square_numbers=[1, 4, 9], cube_numbers=[])


In [4]:
print(f"{DemoModel(square_numbers='1|4|16') = }")

DemoModel(square_numbers='1|4|16') = DemoModel(square_numbers=[1, 4, 16], cube_numbers=[])


In [5]:
print(f"{DemoModel(square_numbers=[16], cube_numbers=[8, 27]) = }")

DemoModel(square_numbers=[16], cube_numbers=[8, 27]) = DemoModel(square_numbers=[16], cube_numbers=[8, 27])


In [6]:
try:
    print(f"{DemoModel(square_numbers=[1, 4, 2]) = }")
except ValidationError as e:
    print(e)

1 validation error for DemoModel
square_numbers -> 2
  2 is not a square number (type=assertion_error)


In [7]:
try:
    print(f"{DemoModel(cube_numbers=[27, 27]) = }")
except ValidationError as e:
    print(e)

1 validation error for DemoModel
cube_numbers
  sum of numbers greater than 42 (type=value_error)


A few more things to note:

* a single validator can be applied to multiple fields by passing it multiple field names.

* a single validator can also be called on all fields by passing the special value "*".

* the keyword argument `pre` will cause the validator to be called prior to other validation.

* passing `each_item=True` will result in the validator being applied to individual values (e.g. of `List`, `Dict`, `Set`, etc.), rather than the whole object.